In [ ]:
# To run shell commands in Jupyter notebook, use the ! prefix or %% magic commands
# Here's how to fix it:

!conda create -n sentiment_analysis python=3.9 -y  # -y flag to automatically answer yes to prompts

# To activate an environment within a notebook, you'd typically need to restart the kernel
# after installation, but for documentation purposes:
!conda activate sentiment_analysis  # Note: this might not work as expected in a notebook

# Install required packages
!conda install tensorflow -y
!conda install pandas numpy matplotlib scikit-learn -y
!pip install kagglehub

Jupyter detected...
2 channel Terms of Service accepted
doneieving notices: - 
Channels:
 - defaults
Platform: osx-64
doneng package metadata (repodata.json): 
doneing environment: - 


==> WARNING: A newer version of conda exists. <==
    current version: 25.5.1
    latest version: 25.7.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /opt/anaconda3/envs/sentiment_analysis

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pip-25.2                   |     pyhc872135_0         1.2 MB
    python-3.9.23              |       hd8516d5_0        12.6 MB
    setuptools-78.1.1          |   py39hecd8cb5_0         1.6 MB
    wheel-0.45.1               |   py39hecd8cb5_0         117 KB
    ------------------------------------------------------------
                               

**Step 1: Download and Load the Dataset**

In [6]:
# Import all required libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import kagglehub

# Check versions
print(f"TensorFlow version: {tf.__version__}")
print(f"Pandas version: {pd.__version__}")

# Download the dataset
print("Downloading dataset...")
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")
print(f"Path to dataset files: {path}")

# Load the dataset
df = pd.read_csv(path + '/IMDB Dataset.csv')
print(f"Dataset shape: {df.shape}")
print("\nFirst few rows:")
print(df.head())
print("\nSentiment distribution:")
print(df['sentiment'].value_counts())

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.45.1 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.
ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


ModuleNotFoundError: No module named 'tensorflow'

In [2]:
# First, install the kagglehub package
!pip install kagglehub

# Now import the required libraries
import kagglehub
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# Download the dataset
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")
print("Path to dataset files:", path)

# Load the dataset
df = pd.read_csv(path + '/IMDB Dataset.csv')
print(f"Dataset shape: {df.shape}")
print(df.head())

ModuleNotFoundError: No module named 'tensorflow'

**Step 2: Explore and Preprocess the Data**

In [2]:
# Check basic info
print(df.info())
print(df['sentiment'].value_counts())

# Convert sentiments to numerical labels
label_encoder = LabelEncoder()
df['sentiment_label'] = label_encoder.fit_transform(df['sentiment'])

# Check the mapping
print("Label mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

# Basic text cleaning
def clean_text(text):
    text = text.lower()
    text = text.replace('<br />', ' ')  # Remove HTML tags
    return text

df['cleaned_review'] = df['review'].apply(clean_text)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
None
sentiment
positive    25000
negative    25000
Name: count, dtype: int64
Label mapping: {'negative': np.int64(0), 'positive': np.int64(1)}


**Step 3: Split the Dataset**

In [3]:
# Split the data into train, validation, and test sets
X = df['cleaned_review']
y = df['sentiment_label']

# First split: 80% train, 20% temp
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Second split: 50% validation, 50% test (from the 20%)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

Training set: 40000 samples
Validation set: 5000 samples
Test set: 5000 samples


**Step 4: Tokenization and Sequence Preparation**

In [4]:
# Tokenize the text
vocab_size = 10000  # Most frequent 10,000 words
max_length = 200    # Maximum sequence length
oov_token = "<OOV>" # Out-of-vocabulary token

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

# Convert texts to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to ensure uniform length
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

print(f"Training sequences shape: {X_train_pad.shape}")
print(f"Validation sequences shape: {X_val_pad.shape}")
print(f"Test sequences shape: {X_test_pad.shape}")

Training sequences shape: (40000, 200)
Validation sequences shape: (5000, 200)
Test sequences shape: (5000, 200)


**Step 5: Build the LSTM Model**

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Preprocessing
label_encoder = LabelEncoder()
df['sentiment_label'] = label_encoder.fit_transform(df['sentiment'])

def clean_text(text):
    text = text.lower()
    text = text.replace('<br />', ' ')
    return text

df['cleaned_review'] = df['review'].apply(clean_text)

# Split data
X = df['cleaned_review']
y = df['sentiment_label']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Tokenization
vocab_size = 10000
max_length = 200
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=max_length, padding='post', truncating='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post', truncating='post')

print(f"Training data shape: {X_train_pad.shape}")
print(f"Validation data shape: {X_val_pad.shape}")

# Create and build model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPooling1D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam

def create_lstm_model(vocab_size, embedding_dim=128, lstm_units=64):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)),
        GlobalMaxPooling1D(),
        Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.3),
        Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    return model

# Create model
model = create_lstm_model(vocab_size)

# Build model with input shape
model.build(input_shape=(None, max_length))

# Compile
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Now summary will work properly
print("Model Summary:")
model.summary()

# Train the model
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

history = model.fit(
    X_train_pad,
    y_train,
    epochs=5,
    batch_size=128,
    validation_data=(X_val_pad, y_val),
    callbacks=[early_stopping],
    verbose=1
)

Training data shape: (40000, 200)
Validation data shape: (5000, 200)
Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 200, 128)       │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,389,185 (5.30 MB)

 Trainable params: 1,389,185 (5.30 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
 94/313 ━━━━━━━━━━━━━━━━━━━━ 3:08 860ms/step - accuracy: 0.5434 - loss: 9.3220

**Step 7: Evaluate the Model**

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test_pad, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Make predictions
y_pred_proba = model.predict(X_test_pad)
y_pred = (y_pred_proba > 0.5).astype(int)

# Additional metrics
from sklearn.metrics import classification_report, confusion_matrix

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

**Step 8: Plot Training History**

In [ ]:
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()

    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history)

**Step 9: Save the Model and Tokenizer**

In [ ]:
# Save the trained model
model.save('sentiment_analysis_lstm_model.h5')

# Save the tokenizer
import pickle
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

print("Model and tokenizer saved successfully!")

**Step 10: Function for Predicting New Reviews**

In [ ]:
def predict_sentiment(text, model, tokenizer, max_length=200):
    """
    Predict sentiment for a new review
    """
    # Clean the text
    cleaned_text = clean_text(text)

    # Convert to sequence and pad
    sequence = tokenizer.texts_to_sequences([cleaned_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length, padding='post', truncating='post')

    # Make prediction
    prediction = model.predict(padded_sequence, verbose=0)[0][0]
    sentiment = 'positive' if prediction > 0.5 else 'negative'
    confidence = prediction if prediction > 0.5 else 1 - prediction

    return sentiment, confidence, prediction

# Test the prediction function
sample_review = "This movie was absolutely fantastic! The acting was superb and the storyline was engaging."
sentiment, confidence, score = predict_sentiment(sample_review, model, tokenizer)
print(f"Review: {sample_review}")
print(f"Predicted sentiment: {sentiment}")
print(f"Confidence: {confidence:.4f}")
print(f"Raw score: {score:.4f}")